<a href="https://colab.research.google.com/github/Nskoushik26276/Product-Pulse-AI/blob/main/product_bot_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip uninstall -y googletrans googletrans-py httpx
!pip install httpx==0.28.1
!pip install flask flask-ngrok pyngrok transformers torch \
deep-translator beautifulsoup4 requests pytrends googlesearch-python \
pandas numpy scikit-learn openai

Found existing installation: httpx 0.28.1
Uninstalling httpx-0.28.1:
  Successfully uninstalled httpx-0.28.1
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)


In [26]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/ngrok
!chmod +x /usr/local/bin/ngrok
!ngrok version

--2025-12-10 04:15:42--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 75.2.60.68, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11284206 (11M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.3’

ngrok-v3-stable-lin 100%[===================>]  10.76M  4.58MB/s    in 2.3s    

2025-12-10 04:15:45 (4.58 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.3’ saved [11284206/11284206]

ngrok
ngrok version 3.34.0


In [27]:
from pyngrok import ngrok
import getpass
ngrok_token = getpass.getpass("🔐 Enter your Ngrok Auth Token → ")
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
    print("✔ Ngrok configured successfully!")
else:
    print("⚠ No token entered. Ngrok may not provide a public URL.")

🔐 Enter your Ngrok Auth Token → ··········
✔ Ngrok configured successfully!


In [32]:
###############################################################################
# ████████ TrendScope-AI : Product Success Predictor + Flask Web UI (Colab) ███████
###############################################################################

# ----------------------------- INSTALLATION ------------------------------
!pip install flask transformers torch scikit-learn requests beautifulsoup4 numpy pandas pyngrok --quiet

import os, re, time, numpy as np, pandas as pd, getpass, requests
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from flask import Flask, request, render_template
from pyngrok import ngrok, conf

# ----------------------------- HIDE OPENAI KEY (OPTIONAL) ----------------
key = getpass.getpass("🔐 OpenAI Key (Optional) → Press Enter to Skip: ")
if key:
    os.environ["OPENAI_API_KEY"] = key
    print("✔ API key secured!\n")

# ----------------------------- NGROK SETUP -------------------------------
ngrok_path = "/usr/local/bin/ngrok"  # ngrok v3 binary path
ngrok_auth = getpass.getpass("🔑 Enter your ngrok Auth Token: ")
ngrok.set_auth_token(ngrok_auth)

ngrok_config = conf.PyngrokConfig(ngrok_path=ngrok_path)
print("✔ Ngrok configured successfully!")

# ----------------------------- 1) SENTIMENT ANALYZER ---------------------
sentiment_model = pipeline("sentiment-analysis")

def extract_keywords(reviews):
    vec = CountVectorizer(stop_words='english')
    try:
        X = vec.fit_transform(reviews)
        words = vec.get_feature_names_out()
        freq = np.array(X.sum(axis=0)).flatten()
        return [x for _, x in sorted(zip(freq, words), reverse=True)]
    except:
        return []

def analyze_reviews(reviews):
    results = []
    for review in reviews:
        try:
            r = sentiment_model(review)[0]
            score = r['score'] if r['label'] == "POSITIVE" else -r['score']
            results.append(score)
        except:
            continue
    avg_score = np.mean(results) if results else 0.0
    return {
        "avg_sentiment_score": avg_score,
        "count": len(results),
        "top_negative_keywords": extract_keywords(reviews)[:5]
    }

# ----------------------------- 2) AMAZON REVIEW SCRAPER ------------------
def auto_collect_reviews(product_name, max_pages=3, per_page_limit=5):
    headers = {"User-Agent":"Mozilla/5.0"}
    reviews = []
    for page in range(1, max_pages+1):
        url = f"https://www.amazon.in/s?k={product_name.replace(' ','+')}+reviews&page={page}"
        html = requests.get(url, headers=headers).text
        rev = re.findall(r'\"reviewText\":\"(.*?)\"', html)
        for r in rev[:per_page_limit]:
            reviews.append(r)
        time.sleep(0.5)
    return reviews, (len(reviews) > 15)

# ----------------------------- 3) GOOGLE TREND APPROX --------------------
def get_trend_data(product_name):
    try:
        url = f"https://trends.google.com/trends/api/explore?hl=en&q={product_name}"
        data = requests.get(url).text
        nums = re.findall(r'\d+', data)
        trend = [int(x) for x in nums[:12]]
        return pd.DataFrame({"month": range(1, len(trend)+1), "score": trend})
    except:
        return None

def compute_trend_slope(df):
    if df is None or len(df) < 3:
        return 0.0
    return np.polyfit(df['month'], df['score'], 1)[0] / 100

# ----------------------------- 4) SUCCESS SCORE --------------------------
def compute_success_score(trend, sentiment, reviews, price):
    score = 0.2*trend + 0.4*sentiment + 0.2*(min(reviews/50,1)) + 0.2*price
    return max(min(score,1),0), {"trend":trend,"sentiment":sentiment,"reviews":reviews,"price":price}

# ----------------------------- 5) FAILURE REASONS + FIXES ----------------
def generate_failure_reason_and_mods(product, review_data, comp):
    reasons, suggestions = [], []
    if comp['sentiment'] < 0.2:
        reasons.append("Customers dislike similar products in the market.")
        suggestions.append("Enhance quality & highlight strong value.")
    if comp['trend'] < 0:
        reasons.append("Popularity is decreasing over time.")
        suggestions.append("Focus on niche branding & influencer awareness.")
    if comp['reviews'] < 20:
        reasons.append("Low user engagement and visibility.")
        suggestions.append("Promote early campaigns & gather more feedback.")
    return reasons, suggestions

# ----------------------------- 6) SUCCESS STRATEGY -----------------------
def generate_success_strategy(name, desc, target_users, regions, trend, review):
    recs = [
        "Launch on Amazon + collaborate with influencers.",
        "Improve based on negative keywords: " + ", ".join(review['top_negative_keywords']),
        "Price competitively & provide strong warranty."
    ]
    if target_users:
        recs.append("Primary Target Customers: " + target_users)
    else:
        recs.append("Suggested Customers: Students, E-commerce buyers, Youth, Tech-savvy consumers.")
    return {"key_recommendations": recs}

# ----------------------------- 7) FLASK WEB APP --------------------------
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    product_name = request.form['product_name']
    desc = request.form['product_desc']
    target_users = request.form.get('target_users', '')
    price_input = request.form.get('price_score', '')
    price_score = float(price_input) if price_input else 0.6

    reviews, product_found = auto_collect_reviews(product_name)
    trend_df = get_trend_data(product_name)
    slope = compute_trend_slope(trend_df)
    review_analysis = analyze_reviews(reviews)

    if product_found:
        return render_template("result.html", result="taken", name=product_name)

    score, comp = compute_success_score(slope, review_analysis['avg_sentiment_score'], review_analysis['count'], price_score)

    if score < 0.5:
        reasons, suggestions = generate_failure_reason_and_mods({"name": product_name, "desc": desc}, review_analysis, comp)
        return render_template("result.html", result="fail", score=round(score*100,1),
                               name=product_name, desc=desc, target_users=target_users,
                               reasons=reasons, suggestions=suggestions)
    else:
        strategy = generate_success_strategy(product_name, desc, target_users, "", slope, review_analysis)
        return render_template("result.html", result="success", score=round(score*100,1),
                               name=product_name, desc=desc, target_users=target_users,
                               strategy=strategy, neg=review_analysis['top_negative_keywords'])

# ----------------------------- 8) HTML TEMPLATES -------------------------
os.makedirs("templates", exist_ok=True)

open("templates/index.html","w").write("""
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/milligram/1.4.1/milligram.min.css">
<h1 style='color:purple;'>🔮 TrendScope AI – Product Predictor</h1>
<form action="/predict" method="POST" style='margin:30px'>
  <label>Product Name:</label><input type="text" name="product_name" required>
  <label>Description:</label><textarea name="product_desc" required></textarea>
  <label>Target Customers (optional):</label><input type="text" name="target_users">
  <label>Price Competitiveness (0-1, optional):</label>
  <input type="text" name="price_score" placeholder="0.6 (default)">
  <button type="submit" class="button">Predict 🚀</button>
</form>
""")

open("templates/result.html","w").write("""
{% if result == "taken" %}
<h2 style="color:red;">⚠️ Product Already Exists on Market!</h2>
<a href="/">⬅ Try Another</a>

{% elif result == "fail" %}
<h2 style="color:red;">🔴 Failure Prediction – Score: {{ score }}%</h2>
{% if target_users %}<p>👥 Target Users: {{ target_users }}</p>{% endif %}
<h3>Reasons:</h3><ul>{% for r in reasons %}<li>{{ r }}</li>{% endfor %}</ul>
<h3>Suggestions:</h3><ul>{% for s in suggestions %}<li>{{ s }}</li>{% endfor %}</ul>
<a href="/">⬅ Try Again</a>

{% elif result == "success" %}
<h2 style="color:green;">🟢 Success Likely – Score: {{ score }}%</h2>
{% if target_users %}<p>👥 Target Users: {{ target_users }}</p>{% endif %}
<h3>Recommended Strategy:</h3><ul>{% for s in strategy.key_recommendations %}<li>{{ s }}</li>{% endfor %}</ul>
<h3>Improve 🔧:</h3><ul>{% for n in neg %}<li>{{ n }}</li>{% endfor %}</ul>
<a href="/">⬅ Predict Again</a>
{% endif %}
""")

# ----------------------------- 9) START NGROK + FLASK ---------------------
print("🌍 Starting TrendScope AI Web App... 🚀")
public_url = ngrok.connect(5000, pyngrok_config=ngrok_config)
print(f"🌐 Open this link in browser: {public_url}")
app.run(port=5000)

🔐 OpenAI Key (Optional) → Press Enter to Skip: ··········
✔ API key secured!

🔑 Enter your ngrok Auth Token: ··········


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


✔ Ngrok configured successfully!


Device set to use cuda:0



🌍 Starting TrendScope AI Web App... 🚀
🌐 Open this link in browser: http://nonlymphatic-nelia-unsurprising.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Dec/2025 04:24:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2025 04:24:55] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Dec/2025 04:25:49] "POST /predict HTTP/1.1" 200 -
